In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from avcv.all import *
import torch, torch.nn as nn
from dms_drowsiness.video_writer import Board
import onnxruntime
import numpy as np, cv2
from PIL import Image
from ple.all import *
import torch.utils.data as td


In [ ]:
class Label:
    def __init__(self, label, video):
        self.label = label
        self.video = video
        
    def check_action_at_frame_idx(self, i):
        if isinstance(i, int):
            i = i/self.video.fps
            
        actions = []
        for action in self.label['annotation']['actionAnnotationList']:
            if i >= action['start'] and i < action['end']:
                action_idx = action['action']
                action_name = self.actionid2name[action_idx]
                actions.append(action_name)
        return actions
    @property
    def actionid2name(self):
        if hasattr(self, '_actionid2name'):
            return self._actionid2name
        ret = dict()
        for actionLabel in self.label['config']['actionLabelData']:
            ret[actionLabel['id']] = actionLabel['name']
        self._actionid2name = ret
        return ret

In [ ]:
PRED_DIR_COCO ='/home/anhvth8/gitprojects/YOLOX/.cache/raw_video_predict_face_food'

## Read 2d video feat

In [ ]:
def convert_flatten_to_2d_feature(flatten_sample):
    feat_sizes = [416//8, 416//16, 416//32]
    # reg_orig_shape = np.array(reg_orig_shape)**2
    cur_i = 0
    feats = []
    for feat_size in feat_sizes:
        a = cur_i
        b = a+feat_size**2
        cur_i = b
        feats.append(flatten_sample[a:b].reshape(feat_size, feat_size, -1))
    return feats

def read_raw_feat_one_video(path):
    data = dict(mmcv.load(path))
    # for k in data:
    #     data[k] = convert_flatten_to_2d_feature(data[k])
    return data
    # list_json_paths[0]['raw_feat_path']

In [ ]:
VIDEO_RAW_DIR_FOOD = '/data/DMS_Behavior_Detection/RawVideos/Action_Eating/'
VIDEO_RAW_DIR_CIGARRET = '/data/DMS_Behavior_Detection/mobile_cigarret_foreignerUS'
VAL_USERS = ['hungng', 'chungtd12', 'thomp4', 'thuyhv5']

### get_meta_jsons

109

In [ ]:
json_paths = list_json_paths[0]
cc = CocoDataset(json_paths['pred_json_path'])

### video_to_coco_format

In [ ]:
def construct_cmd(video_inp, dry_run=True):
    cmd=f"python tools/eval_export.py -f exps/dms/mb2_face_food.py --input_video {video_inp} --out_dir .cache/raw_video_predict_face_food/ -d 1"
    if not dry_run:
        os.system(cmd)
    return cmd

def video_to_coco_format(tobe_extract_videos):


    if len(tobe_extract_videos):
        with open('/tmp/listcmd.sh', 'w') as f:
            for video in tobe_extract_videos:
                cmd = construct_cmd(video)
                f.write(cmd+'\n')
        !python scripts/run_list_commands.py /tmp/listcmd.sh 8 

    else:
        print('All video are extracted to coco-format :D')
        
video_to_coco_format(tobe_extract_videos)

All video are extracted to coco-format :D


# Extract coco-format prediction of detection model

In [ ]:
def collect_data(inp):
    index, json_paths = inp
    v = mmcv.VideoReader(json_paths['video_path'])
    label = Label(mmcv.load(json_paths['label_path']), v)
    cc = CocoDataset(json_paths['pred_json_path'])
    data = []
    for i in cc.img_ids:
        actions = label.check_action_at_frame_idx(i)
        anns = cc.gt.imgToAnns[i]
        img = cc.gt.imgs[i]
        img_path = osp.join(cc.img_dir, img['file_name'])
        data.append((actions[0] if len(actions) else 'none' , index, img_path, img['id']))
    return data


In [ ]:
%%time

CACHE_DF_PATH = '/tmp/eating_cache_data_df.pkl'
if not osp.exists(CACHE_DF_PATH) or 1:
    list_json_paths = []
    label_paths  = glob('/data/DMS_Behavior_Detection/RawVideos/Action_Eating/**/*.json', recursive=True)
    for label_path in label_paths:
        try:
            json_paths = get_meta_jsons(label_path,)
            list_json_paths.append(json_paths)
        except Exception as e:
            print(e)
            pass

    label_paths  = glob('/data/DMS_Behavior_Detection/mobile_cigarret_foreignerUS/*/**/*.json', recursive=True)
    print(f'{len(label_paths)=}')
    for label_path in label_paths:
        try:
            json_paths = get_meta_jsons(label_path,)
            list_json_paths.append(json_paths)
        except Exception as e:
            print(e)
            pass
    len(list_json_paths)    
    
    
    print('Collecting data from scratch')
    data = multi_thread(collect_data, list(enumerate(list_json_paths)), verbose=1, pbar_iterval=10)
    all_data = []
    for _ in data: all_data += _

    df = pd.DataFrame(all_data, columns=['action', 'video_index', 'img_path', 'img_id'])
    #-smocking->smoking
    ids = df[df['action'] == 'smocking'].index
    df.loc[ids, 'action'] = 'smoking'
    mmcv.dump(df, CACHE_DF_PATH)
else:
    print(f'Load data from {CACHE_DF_PATH=}')
    df = mmcv.load(CACHE_DF_PATH)
# df = df.set_index('img_path')
img_path2index = dict({v:k for k, v in enumerate(sorted(df.img_path.tolist()))})
df['img_idx'] = df['img_path'].apply(lambda path: img_path2index[path])
df = df.set_index('img_idx')

2022-09-28 12:34:13.360 | INFO     | avcv.process:multi_thread:25 - 


len(label_paths)=67
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 110/109, 10.2 task/s, elapsed: 11s, ETA:     0s

2022-09-28 12:34:24.198 | INFO     | avcv.process:multi_thread:34 - multi_thread collect_data, 


CPU times: user 12.1 s, sys: 1.97 s, total: 14.1 s
Wall time: 11.1 s


In [ ]:
df['username'] = df.img_path.apply(lambda x:x.split('/')[-3].split('_')[0])
df['is_val'] = df['username'].apply(lambda x: x in VAL_USERS)

In [ ]:
_id2action = dict(enumerate(df['action'].apply(str).unique().tolist()))
action2id = {v:k for k, v in _id2action.items()}
id2action = {k:v for k, v in _id2action.items()}
print(action2id)

{'eating': 0, 'none': 1, 'mobile usage': 2, 'smoking': 3}


## Split train/val

In [ ]:
catname2id = {cat['name']:cat['id'] for cat in cc.gt.cats.values()}

In [ ]:
catname2id

{'face': 0, 'mouth': 3, 'phone': 4, 'cigarette': 5, 'food/drink': 6, 'eye': 1}

In [ ]:
def get_max_score_ann(anns, catname):
    cat_id = catname2id[catname]
    anns_cat = [ann for ann in anns if ann['category_id'] == cat_id]
    anns_cat = list(sorted(anns_cat, key=lambda x:x['score']))
    if len(anns_cat):
        return anns_cat[-1]
    return None
    

In [ ]:
imgpath2index = {v:k for k,v in df['img_path'].to_dict().items()}

In [ ]:
objects = ['face', 'mouth', 'phone','cigarette', 'food/drink']
for obj in objects:
    df[obj] = None

In [ ]:
df['img_size'] = df.img_path.apply(lambda x:Image.open(x).size)

In [ ]:
for json_paths in tqdm(list_json_paths):
    cc = CocoDataset(json_paths['pred_json_path'])
    for img_id in cc.img_ids:
        img = cc.gt.imgs[img_id]
        img_path = osp.abspath(osp.join(cc.img_dir, img['file_name']))
        anns = cc.gt.imgToAnns[img_id]
        index = imgpath2index[img_path]
        for obj in objects:
            ann = get_max_score_ann(anns, obj)
            imw, imh = df.loc[index].img_size
            if ann is not None:
                x,y,w,h = ann['bbox']
                x1,y1 = x,y
                x2, y2 = x1+w, y1+h
                df.loc[index, obj] = str([x1/imw,y1/imh,x2/imw,y2/imh,ann['score']])

100%|████████████████████████████████████████████████████████████████████| 109/109 [01:48<00:00,  1.01it/s]


In [ ]:
row = df.iloc[0]

In [ ]:
bops.box_iou??

Object `bops.box_iou` not found.


In [ ]:
import torchvision
import torch
import torchvision.ops.boxes as bops
def xyxys2box(box):
    if isinstance(box, str):
        return eval(box)[:4]
    else:
        return box

In [ ]:
# row['food/drink']

In [ ]:
def visualize_one_row(row):
    img = mmcv.imread(row.img_path)
    box_colors = [        0.000, 0.447, 0.741,
        0.850, 0.325, 0.098,
        0.929, 0.694, 0.125,
        0.494, 0.184, 0.556,
        0.466, 0.674, 0.188,
        0.301, 0.745, 0.933,
        0.635, 0.078, 0.184,
        0.300, 0.300, 0.300,]
    box_colors = np.array(box_colors).reshape(-1, 3)*255
    box_colors = box_colors.astype(int).tolist()
    imw, imh = row.img_size
    
    for i, obj in enumerate(objects):
        if row[obj] is None: continue
        x1,y1,x2,y2,s = eval(row[obj])
        x1 *= imw
        x2 *= imw
        y1 *= imh
        y2 *= imh
        cls_ids = [0]
        texts = [f'{obj} {s*100:0.2f}%']
        bboxes = [[x1,y1,x2,y2]]
        scores = [s]
        img = bbox_visualize(img, bboxes, scores,cls_ids, texts=texts, box_color=box_colors[i], conf=0.0)

    imshow(img, dpi=50)

In [ ]:

def cal_iou(box1, box2):
    box1 = xyxys2box(box1)
    box2 = xyxys2box(box2)
    box1 = torch.tensor([box1], dtype=torch.float)
    box2 = torch.tensor([box2], dtype=torch.float)
    iou = bops.generalized_box_iou(box1, box2).item()
    return iou
cal_iou(row.mouth, row.face, ),cal_iou(row.face, row.mouth, )

def cal_bbox_overlap(bb1, bb2, over_box_2=True):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    def to_dict(bb):
        bb=xyxys2box(bb)
        x1,y1,x2,y2=bb
        return dict(x1=x1, y1=y1,x2=x2,y2=y2)
    if bb1 is None or bb2 is None:
        return None
    bb1 = to_dict(bb1)
    bb2 = to_dict(bb2)
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    if over_box_2:
        bb = bb2
    else:
        bb = bb1
        
    area = (bb['y2']-bb['y1'])*(bb['x2']-bb['x1'])
    return intersection_area/area

In [ ]:
    cx1, cy1 = box1[0::2].mean(), box1[0::1].mean()
    cx2, cy2 = box2[0::2].mean(), box2[0::1].mean()
    distance = ((cx2-cx1)**2+(cy2-cy1)**2)**0.5

In [ ]:
def bbox_distance(bb1, bb2):
    def _get_center(bb):
        bb = xyxys2box(bb)[:4]
        x1,y1,x2,y2 = bb
        return (x1+x2)/2, (y1+y2)/2
    if bb1 is None or bb2 is None: return None

    cx1, cy1 = _get_center(bb1)
    cx2, cy2 = _get_center(bb2)
    return np.sqrt((cx2-cx1)**2+(cy2-cy1)**2)

In [ ]:
df['iou_face_food'] = df.apply(lambda row:cal_bbox_overlap(row['face'], row['food/drink']), 1)
df['iou_face_cigarette'] = df.apply(lambda row:cal_bbox_overlap(row['face'], row['cigarette']), 1)
df['iou_face_phone'] = df.apply(lambda row:cal_bbox_overlap(row['face'], row['phone']), 1)


In [ ]:
df['distance_face_food'] = df.apply(lambda row:bbox_distance(row['face'], row['food/drink']), 1)
df['distance_face_cigarette'] = df.apply(lambda row:bbox_distance(row['face'], row['cigarette']), 1)
df['distance_face_phone'] = df.apply(lambda row:bbox_distance(row['face'], row['phone']), 1)


In [ ]:
def get_score(bb):
    if bb is None:return bb
    s = eval(bb)[-1]
    return s
df['food_score'] = df.apply(lambda row: get_score(row['food/drink']), 1)
df['cigarette_score'] = df.apply(lambda row: get_score(row['cigarette']), 1)
df['phone_score'] = df.apply(lambda row: get_score(row['phone']), 1)

In [ ]:

from sklearn import tree
def construct_x(row, FEATURE_NAMES):
    data = []
    for feature_name in FEATURE_NAMES:
        value = row[feature_name]
        if np.isnan(value):
            if 'distance' in feature_name:
                value = 10
            elif 'score' in feature_name:
                value = 0
        if 'score' in feature_name:
            if value < 0.3:
                value = 0
        data.append(value)
    return data




In [ ]:
ALL_FEATURE_NAMES=['iou_face_food', 'iou_face_cigarette', 'iou_face_phone', 'distance_face_food', 'distance_face_cigarette', 'distance_face_phone', 'food_score', 'cigarette_score', 'phone_score']
def get_features_names(object_name):
    
    FEATURE_NAMES = [_ for _ in ALL_FEATURE_NAMES if object_name in _ and not 'iou' in _]
    return FEATURE_NAMES

In [ ]:

MAX_DEPTH = 2
def build_tree_cls(object_name='phone', class_name='mobile usage'):
    FEATURE_NAMES = get_features_names(object_name)
    print(f'{FEATURE_NAMES=}, {class_name=}')
    def sample_data_frames(df, class_name):
        df = df.copy()
        df_positive = df[df['action'] == class_name]
        df_negative = df[df['action'] != class_name].sample(len(df_positive))
        df_negative['action'] = 'unknown'
        _df = pd.concat([df_positive, df_negative])

        _df['X'] = df.apply(lambda x:construct_x(x, FEATURE_NAMES), 1)

        X = np.array(_df.X.values.tolist())
        y = np.array(_df['action'].tolist())
        return X, y

    
    X,y = sample_data_frames(df, class_name)
    clf = tree.DecisionTreeClassifier(max_depth=MAX_DEPTH)
    clf = clf.fit(X, y)
    return clf

In [ ]:
def predict(df, object_name, class_name):
    clf = clfs[class_name]
    FEATURE_NAMES = get_features_names(object_name)
    print(f'{FEATURE_NAMES=}, {class_name=}')
    
    df['X'] = df.apply(lambda x:construct_x(x, FEATURE_NAMES), 1)

    X = np.array(df.X.values.tolist())
    return clf.predict(X)

In [ ]:
clfs = {}
for object_name, class_name in [('phone', 'mobile usage'), ('food', 'eating'), ('cigarette', 'smoking')]:
    clfs[class_name] = build_tree_cls(object_name, class_name)

FEATURE_NAMES=['distance_face_phone', 'phone_score'], class_name='mobile usage'


/home/anhvth8/.conda/envs/py38/lib/python3.8/site-packages/sklearn/tree/tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)


TypeError: __cinit__() takes exactly 5 positional arguments (6 given)

In [ ]:
df['y_pred_mobile_usage'] = predict(df, 'phone', 'mobile usage')
df['y_pred_eating'] = predict(df, 'food', 'eating')
df['y_pred_smoking'] = predict(df, 'cigarette', 'smoking')

In [ ]:
df[df.action=='eating']['distance_face_food'].max()

In [ ]:
row.phone_score

In [ ]:
row = df[df.action=='eating'].iloc[0]

def is_eating(row, score_thr=0.3, dist=0.5):
    if np.isnan(row.food_score) or np.isnan(row.food_score):return False
        
    if row.food_score > score_thr and row.distance_face_food < dist:
        return True
    return False

def is_using_phone(row, score_thr=0.3, dist=0.5):
    if np.isnan(row.phone_score) or np.isnan(row.distance_face_phone):
        return False
    if row.phone_score > score_thr and row.distance_face_phone < dist:
        return True
    return False

def is_smoking(row, score_thr=0.3, dist=0.5):
    if np.isnan(row.cigarette_score) or np.isnan(row.distance_face_cigarette):
        return False
    if row.cigarette_score > score_thr and row.distance_face_cigarette < dist:
        return True
    return False

def predict_rule(row):
    rt = []
    if is_eating(row):
        rt.append('eating')
        
    if is_using_phone(row):
        rt.append('mobile usage')
        
    if is_smoking(row):
        rt.append('smoking')
        
    return rt
df['action_pred'] = df.apply(predict_rule, 1)

In [ ]:
def draw_one_row(row):
    img = mmcv.imread(row.img_path)
    box_colors = [        0.000, 0.447, 0.741,
        0.850, 0.325, 0.098,
        0.929, 0.694, 0.125,
        0.494, 0.184, 0.556,
        0.466, 0.674, 0.188,
        0.301, 0.745, 0.933,
        0.635, 0.078, 0.184,
        0.300, 0.300, 0.300,]
    box_colors = np.array(box_colors).reshape(-1, 3)*255
    box_colors = box_colors.astype(int).tolist()
    imw, imh = row.img_size
    
    for i, obj in enumerate(objects):
        if row[obj] is None: continue
        x1,y1,x2,y2,s = eval(row[obj])
        x1 *= imw
        x2 *= imw
        y1 *= imh
        y2 *= imh
        cls_ids = [0]
        texts = [f'{obj} {s*100:0.2f}%']
        bboxes = [[x1,y1,x2,y2]]
        scores = [s]
        img = bbox_visualize(img, bboxes, scores,cls_ids, texts=texts, box_color=box_colors[i], conf=0.0)

    return img

In [ ]:
predict_rule(df.loc[82553])

['eating']

In [ ]:
# _df = df[df.video_index==32]

In [ ]:
list_vis = []
for _, row in tqdm(_df[0::10].iterrows(), total=len(_df)):
    vis = draw_one_row(row)
    board = Board(10, line_w=600)
    # for i, action in enumerate(row.action_pred):
    i = 1
    board.set_line_text(0, ' '.join(row.action_pred)+f' | {row.name}')
    
    
    for feat_name in ALL_FEATURE_NAMES:
        val = row[feat_name]
        if not np.isnan(val):
            board.set_line_text(i, f'{feat_name}:{val:0.2f}')
            i+=1
    list_vis += [board.img_concat(vis)]    
images_to_video(list_vis, 'vis.mp4', output_size=(700, 300))

 10%|██████▌                                                           | 214/2132 [00:01<00:10, 184.62it/s]
2022-09-28 16:32:14.603 | INFO     | avcv.utils:images_to_video:267 - Write video, output_size: (700, 300)


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 214/214, 261.0 task/s, elapsed: 1s, ETA:     0s

2022-09-28 16:32:15.425 | INFO     | avcv.utils:images_to_video:277 - -> /home/anhvth8/gitprojects/YOLOX/vis.mp4


In [ ]:
ls .cache/raw_video_predict_face_food/vietph_Sensing_Session2_CAMc_1b_2c_3a_4d_5b_6b_7b_8a_9a_10a_11b_12b_13b_14b_15b_16f_17a_18a_19a_20b/annotations/

images.json  pred_mb2_face_food.json  pred_mb2_face_food_raw_outputs.pkl


In [ ]:
cc = CocoDataset('.cache/raw_video_predict_face_food/vietph_Sensing_Session2_CAMc_1b_2c_3a_4d_5b_6b_7b_8a_9a_10a_11b_12b_13b_14b_15b_16f_17a_18a_19a_20b/annotations/pred_mb2_face_food.json')

In [ ]:
row

action                                                                  none
video_index                                                               32
img_path                   /home/anhvth8/gitprojects/YOLOX/.cache/raw_vid...
img_id                                                                  2130
username                                                              tunm13
is_val                                                                 False
face                       [0.3469236195087433, 0.2684265772501628, 0.558...
mouth                      [0.4113991856575012, 0.6045654084947374, 0.488...
phone                                                                   None
cigarette                                                               None
food/drink                                                              None
img_size                                                          (512, 288)
iou_face_food                                                            NaN

In [ ]:
objects

['face', 'mouth', 'phone', 'cigarette', 'food/drink']

In [ ]:
list_data = []
for img_id in tqdm(cc.img_ids[0::10]):
    data = dict()
    img = cc.gt.imgs[img_id]
    img_path = osp.abspath(osp.join(cc.img_dir, img['file_name']))
    anns = cc.gt.imgToAnns[img_id]
    data['img_path'] = img_path
    # index = imgpath2index[img_path]
    
    for obj in objects:
        ann = get_max_score_ann(anns, obj)
        # imw, imh = df.loc[index].img_size
        data['img_size'] = imw, imh = Image.open(img_path).size
        if ann is not None:
            x,y,w,h = ann['bbox']
            x1,y1 = x,y
            x2, y2 = x1+w, y1+h
            data[obj] = str([x1/imw,y1/imh,x2/imw,y2/imh,ann['score']])
            data[f'{obj}_score'] = ann['score']
    list_data.append(data)

100%|████████████████████████████████████████████████████████████████| 4920/4920 [00:03<00:00, 1410.86it/s]


In [ ]:
df = pd.DataFrame(list_data)

In [ ]:
def bbox_distance(bb1, bb2):
    try:
        def _get_center(bb):
            bb = xyxys2box(bb)[:4]
            x1,y1,x2,y2 = bb
            return (x1+x2)/2, (y1+y2)/2
        if bb1 is None or bb2 is None: return None
        cx1, cy1 = _get_center(bb1)
        cx2, cy2 = _get_center(bb2)
        return np.sqrt((cx2-cx1)**2+(cy2-cy1)**2)
    except:
        return None

In [ ]:
df['distance_face_food'] = df.apply(lambda row:bbox_distance(row['face'], row['food/drink']), 1)
df['distance_face_cigarette'] = df.apply(lambda row:bbox_distance(row['face'], row['cigarette']), 1)
df['distance_face_phone'] = df.apply(lambda row:bbox_distance(row['face'], row['phone']), 1)

In [ ]:

def is_eating(row, score_thr=0.3, dist=0.5):
    if np.isnan(row['food/drink_score']) or np.isnan(row.distance_face_food):return False
    if row['food/drink_score'] > score_thr and row.distance_face_food < dist:
        return True
    return False

def is_using_phone(row, score_thr=0.3, dist=0.5):
    if np.isnan(row.phone_score) or np.isnan(row.distance_face_phone):
        return False
    if row.phone_score > score_thr and row.distance_face_phone < dist:
        return True
    return False

def is_smoking(row, score_thr=0.3, dist=0.5):
    if np.isnan(row.cigarette_score) or np.isnan(row.distance_face_cigarette):
        return False
    if row.cigarette_score > score_thr and row.distance_face_cigarette < dist:
        return True
    return False

def predict_rule(row):
    rt = []
    if is_eating(row):
        rt.append('eating')
        
    if is_using_phone(row):
        rt.append('mobile usage')
        
    if is_smoking(row):
        rt.append('smoking')
        
    return rt
df['action_pred'] = df.apply(predict_rule, 1)


In [ ]:
def draw_one_row(row):
    img = mmcv.imread(row.img_path)
    box_colors = [        0.000, 0.447, 0.741,
        0.850, 0.325, 0.098,
        0.929, 0.694, 0.125,
        0.494, 0.184, 0.556,
        0.466, 0.674, 0.188,
        0.301, 0.745, 0.933,
        0.635, 0.078, 0.184,
        0.300, 0.300, 0.300,]
    box_colors = np.array(box_colors).reshape(-1, 3)*255
    box_colors = box_colors.astype(int).tolist()
    imw, imh = row.img_size
    
    for i, obj in enumerate(objects):
        if not isinstance(row[obj], str): continue
        x1,y1,x2,y2,s = eval(row[obj])
        x1 *= imw
        x2 *= imw
        y1 *= imh
        y2 *= imh
        cls_ids = [0]
        texts = [f'{obj} {s*100:0.2f}%']
        bboxes = [[x1,y1,x2,y2]]
        scores = [s]
        img = bbox_visualize(img, bboxes, scores,cls_ids, texts=texts, box_color=box_colors[i], conf=0.0)

    return img

In [ ]:
list_vis = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    vis = draw_one_row(row)
    board = Board(10, line_w=600)
    # for i, action in enumerate(row.action_pred):
    i = 1
    board.set_line_text(0, ' '.join(row.action_pred)+f' | {row.name}')
    
    
    for feat_name in ALL_FEATURE_NAMES:
        try:
            val = row[feat_name]
            if not np.isnan(val):
                board.set_line_text(i, f'{feat_name}:{val:0.2f}')
                i+=1
        except:pass
    list_vis += [board.img_concat(vis)]    
images_to_video(list_vis, 'vis.mp4', output_size=(700, 300))

100%|█████████████████████████████████████████████████████████████████| 4920/4920 [00:26<00:00, 184.48it/s]
2022-09-28 16:49:40.781 | INFO     | avcv.utils:images_to_video:267 - Write video, output_size: (700, 300)


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4920/4920, 306.5 task/s, elapsed: 16s, ETA:     0s

2022-09-28 16:49:56.846 | INFO     | avcv.utils:images_to_video:277 - -> /home/anhvth8/gitprojects/YOLOX/vis.mp4
